In [48]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import ollama
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
import pinecone
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Pinecone as PC
# Cell 1: Import necessary libraries
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
import pinecone
import os
from dotenv import load_dotenv
import ollama

# Load environment variables
load_dotenv()
api_key_resume = os.getenv("PINECONE_API_KEY_RESUME")
api_key_jd = os.getenv("PINECONE_API_KEY_JD")
GROQ_KEY=os.getenv("GROQ_API_KEY")

In [49]:
print(f"Resume API Key: {api_key_resume}")
print(f"Job Description API Key: {api_key_jd}")
print(GROQ_KEY)



Resume API Key: pcsk_4CEtRb_EYiumo4ypRb3j4oeEVneiLZWa8ZkYg87Ty83LxhfvunoBjDk6yPa3WuWz47PSHK
Job Description API Key: pcsk_7W6gTn_URecqxiU2F4gWmLUsLEozTRqAk97jZ6f5xvknevuRqkwik6LATcyA1tCCTncbN1
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


In [50]:
#!pip install --upgrade pinecone-client

In [51]:
ollama.pull('nomic-embed-text')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [52]:
loader = DirectoryLoader("resume", glob="./*.pdf", loader_cls=PyPDFLoader)

In [53]:
document=loader.load()

In [54]:
document

[Document(metadata={'source': 'resume\\ac_cv.pdf', 'page': 0}, page_content='Amogh Chavan\nData Scientist, Machine Learning Engineer and Front End Developer\n♂¶obile+91 7030790097 auuchavan@gmail.com /h⌢mePanvel, Navi Mumbai\n/linkedinin/amogh-chavan-21-coder//githubgithub.com/ac-coder-21 X x.com/accode21\nProfile\nAs a passionate Machine Learning enthusiast, I possess a comprehensive understanding of various machine learning\nalgorithms, adept at both implementing them using popular libraries like Scikit-learn and TensorFlow, and building them\nfrom scratch. Furthermore, my proficiency in Data Visualization tools such as Matplotlib, Seaborn and Plotly allows me\nto create compelling visualizations that uncover actionable insights from complex datasets. With these skills, I tackle\ndiverse data challenges, from modeling to analysis and clear presentation for informed decision-making.\nAreas of Expertise\nPython - Flask - Django - Machine Learning - EDA - Data Analysis - Data Visualizat

In [55]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [56]:
text_chunks = text_splitter.split_documents(document)

In [57]:
text_chunks

[Document(metadata={'source': 'resume\\ac_cv.pdf', 'page': 0}, page_content='Amogh Chavan\nData Scientist, Machine Learning Engineer and Front End Developer\n♂¶obile+91 7030790097 auuchavan@gmail.com /h⌢mePanvel, Navi Mumbai\n/linkedinin/amogh-chavan-21-coder//githubgithub.com/ac-coder-21 X x.com/accode21\nProfile\nAs a passionate Machine Learning enthusiast, I possess a comprehensive understanding of various machine learning\nalgorithms, adept at both implementing them using popular libraries like Scikit-learn and TensorFlow, and building them\nfrom scratch. Furthermore, my proficiency in Data Visualization tools such as Matplotlib, Seaborn and Plotly allows me\nto create compelling visualizations that uncover actionable insights from complex datasets. With these skills, I tackle\ndiverse data challenges, from modeling to analysis and clear presentation for informed decision-making.\nAreas of Expertise\nPython - Flask - Django - Machine Learning - EDA - Data Analysis - Data Visualizat

In [58]:
embedding = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [59]:
len(embedding.embed_query("How are you"))

768

In [60]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from pinecone import Pinecone

   # Initialize Pinecone client
pc_resume = Pinecone(api_key=api_key_resume)
pc_jd = Pinecone(api_key=api_key_jd)


In [61]:
print(pc_resume)

In [62]:
index_name = "resume-index"

In [63]:


docsearch = PC.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

PineconeConfigurationError: You haven't specified an Api-Key.

In [ ]:
query = "What are Allergies"

docs = docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='Description\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, ormore than one in every five people, suffer from someform of allergy, with similar proportions throughoutmuch of the rest of the world. Allergy is the single largestreason for school absence and is a major source of lostproductivity in the workplace.\nAn allergy is a type of immune reaction. Normally,'), Document(page_content='Description\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, ormore than one in every five people, suffer from someform of allergy, with similar proportions throughoutmuch of the rest of the world. Allergy is the single largestreason for school absence and is a major source of lostproductivity in the workplace.\nAn allergy is a type of immune reaction. Normally,'), Document(page_content='Allergen —A substance that provokes an allergic\nresponse.\nAllergic rhinitis —

In [ ]:
retriever = docsearch.as_retriever(search_kwargs={'k':2})

In [ ]:
prompt_template="""
    Use the following pieces of information to answer the user's question.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: {context}
    Question: {question}

    Only return the helpful answer below and nothing else.
    Helpful answer:
    """

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=ChatGroq(temperature=0.5, groq_api_key=GROQ_KEY, model_name="llama-3.1-70b-versatile")

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa.invoke({'query': user_input})
  print(f"Answer: {result['result']}")

d:\Programming\Foundations of Gen AI\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Answer: Allergy is a reaction of the immune system.
Answer: Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Answer: Bile duct cancer, also known as cholangiocarcinoma, is a malignant tumor of the bile ducts within the liver (intrahepatic) or leading from the liver to the small intestine (extrahepatic).
Answer: Bile duct cancer, also known as cholangiocarcinoma, is a malignant tumor of the bile ducts within the liver (intrahepatic) or leading from the liver to the small intestine (extrahepatic).
Exiting


SystemExit: 

d:\Programming\Foundations of Gen AI\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
